Import the important libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

[](https://www.kaggle.com/iabhishekofficial/mobile-price-classification)

In [ ]:
df_train=pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
df_train.head()

Some Exploratory data analysis

In [ ]:
df_train.columns

In [ ]:
df_train.info

explore the target variable

In [ ]:
df_train['price_range'].describe()

In [ ]:
sns.distplot(df_train['price_range'])

In [ ]:
df_train['price_range'].hist()

In [ ]:
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(18, 14))
sns.heatmap(corrmat, square=True,annot=True);

From the heatmap above,we are going to explore the variables that are highly correlated to the target variable

In [ ]:
var='battery_power'
data = pd.concat([df_train['price_range'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='price_range', ylim=(0,4));


In [ ]:

var='blue'
data = pd.concat([df_train['price_range'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='price_range', ylim=(0,4));


In [ ]:
var='clock_speed'
data = pd.concat([df_train['price_range'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='price_range', ylim=(0,4));


In [ ]:
var='ram'
data = pd.concat([df_train['price_range'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='price_range', ylim=(0,4));


Exploration of the variables is inconclusive except for the heatmap so I'm going to engineer some features.
-from the heatmap above px_height and px_width are highly correlated so I'm going to combine them into one variable called px_area

-sc_h and sc_w are I'm going to do the same and create a variable called phone area

In [ ]:
df_train['px_area']= df_train['px_height']* df_train['px_width']

In [ ]:
df_train['phone_area']= df_train['sc_h']* df_train['sc_w']

I'm now going to drop them variables from the train data frame

In [ ]:
train_data=df_train.drop(['px_height','px_width','sc_h','sc_w'],axis=1)


Can now view my new train_data

In [ ]:
train_data

Let me now upload my test Data and fine tune it to fit the train data

In [ ]:
df_test=pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')

Adding the px_area and phone_area features

In [ ]:
df_test['px_area']= df_test['px_height']*df_test['px_width']
df_test['phone_area']= df_train['sc_h']* df_train['sc_w']
df_test

Dropping the unwanted columns

In [ ]:
test_data=df_test.drop(['px_height','px_width','sc_h','sc_w','id'],axis=1)
test_data

In [ ]:
#modelling


before we model, we have to establish our target variable which is the price_range

In [ ]:
y=train_data['price_range']

We can now drop it from the training data so as to avoid data leakage and over fitting

In [ ]:

X=train_data.drop('price_range',axis=1)


In [ ]:
X

Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

In [ ]:
#===LOGISTIC REGRESSION=====
from sklearn.linear_model import LogisticRegression
logmodel=LogisticRegression()

In [ ]:
logmodel.fit(X_train,y_train)

In [ ]:
logmodel.score(X_train,y_train)

In [ ]:
logmodel.score(X_test,y_test)

Logistic regression perfroms terribly, so we are going to try other models.. lets start with linear regresion

In [ ]:
#=====LINEAR REGRESSION=======
from sklearn.linear_model import LinearRegression
lm=LinearRegression()

In [ ]:
lm.fit(X_train,y_train)

In [ ]:
lm.score(X_train,y_train)

In [ ]:
lm.score(X_test,y_test)

Linear regression performs not too bad, but lets try other models
KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)

In [ ]:
knn.score(X_train,y_train)

In [ ]:
knn.score(X_test,y_test)

Terrible!!!!!  lets try the good ol random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train,y_train)

In [ ]:
rfc.score(X_train,y_train)

Overfitting??...lets explore the score on the test set and see

In [ ]:
rfc.score(X_test,y_test)

yes, there are signs of overfitting because the random forest performed considerably well on the train data but performed not so good on the unseen test data

From the 4 models above, linear regression performed best on both the train and test data, so its what we will use to predict the test data price_range

In [ ]:
predictions=lm.predict(test_data)
predictions

terrible predictions, we needed 0,1,2 0r 3...we will fine tune it later. For now lets use possibly overfit random forest 

In [ ]:
RF_pred=rfc.predict(test_data)
RF_pred

perfect, lets now append the predicted values to the test_data

In [ ]:
test_data['price_range']=RF_pred

Viewing the test data now

In [ ]:
test_data